In [38]:
%cd /data/bruingjde/on-going/SNAM2021-code/

import os

import joblib
import networkx as nx
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

/data/bruingjde/on-going/SNAM2021-code


In [45]:
class ProgressParallel(joblib.Parallel):
  def __init__(self, use_tqdm=True, total=None, desc=None, unit='it', *args, 
               **kwargs):
    self._use_tqdm = use_tqdm
    self._total = total
    self._desc = desc
    self._unit = unit
    super().__init__(*args, **kwargs)

  def __call__(self, *args, **kwargs):
    with tqdm(disable=not self._use_tqdm, total=self._total, 
              desc=self._desc, unit=self._unit, leave=False, 
              miniters=1) as self._pbar:
      return joblib.Parallel.__call__(self, *args, **kwargs)

  def print_progress(self):
    if self._total is None: 
      self._pbar.total = self.n_dispatched_tasks
    self._pbar.n = self.n_completed_tasks
    self._pbar.refresh()
    
def number_of_events(idx):
  filepath = os.path.join('data', f'{idx:02}', 'edgelist.pkl')
  print(f'{filepath=}')
  edgelist = (
    pd.read_pickle(filepath)
    .loc[lambda x: x['source'] != x['target'], ['source', 'target']]
  )
  return {
    'idx': idx,
    'average events per edge': len(edgelist) / len(edgelist.drop_duplicates())
  }

In [48]:
data = ProgressParallel(n_jobs=30, total=30)(
  joblib.delayed(number_of_events)(idx) for idx in np.arange(1, 31)
)

  0%|          | 0/30 [00:00<?, ?it/s]

In [57]:
pd.DataFrame.from_records(data, index='idx').sort_values('average events per edge')

,average events per edge
idx,
15,1.000000
17,1.000000
16,1.000000
18,1.000000
8,1.000000
10,1.000000
4,1.000000
27,1.000000
20,1.000000
